In [1]:
import json

In [5]:
train_data = []
with open('../dataset/PhoNER/train_word.json', 'r', encoding='utf-8') as f:
    for line in f:
        train_data.append(json.loads(line.strip()))

print(f"Tổng số câu: {len(train_data)}")
print(f"\nVí dụ một câu:")
print(json.dumps(train_data[0], ensure_ascii=False, indent=2))

Tổng số câu: 5027

Ví dụ một câu:
{
  "words": [
    "Đồng_thời",
    ",",
    "bệnh_viện",
    "tiếp_tục",
    "thực_hiện",
    "các",
    "biện_pháp",
    "phòng_chống",
    "dịch_bệnh",
    "COVID",
    "-",
    "19",
    "theo",
    "hướng_dẫn",
    "của",
    "Bộ",
    "Y_tế",
    "."
  ],
  "tags": [
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "O",
    "B-ORGANIZATION",
    "I-ORGANIZATION",
    "O"
  ]
}


In [6]:
print("=" * 60)
print("CẤU TRÚC DATASET")
print("=" * 60)

# Kiểm tra keys trong mỗi entry
sample = train_data[0]
print(f"\nKeys trong mỗi entry: {list(sample.keys())}")
print(f"\nVí dụ entry đầu tiên:")
print(f"  - Số từ: {len(sample['words'])}")
print(f"  - Số nhãn: {len(sample['tags'])}")
print(f"  - Words: {sample['words'][:10]}...")
print(f"  - Tags: {sample['tags'][:10]}...")

# Kiểm tra độ dài câu
lengths = [len(item['words']) for item in train_data]
print(f"\nThống kê độ dài câu:")
print(f"  - Min: {min(lengths)}")
print(f"  - Max: {max(lengths)}")
print(f"  - Mean: {sum(lengths)/len(lengths):.2f}")
print(f"  - Median: {sorted(lengths)[len(lengths)//2]}")

CẤU TRÚC DATASET

Keys trong mỗi entry: ['words', 'tags']

Ví dụ entry đầu tiên:
  - Số từ: 18
  - Số nhãn: 18
  - Words: ['Đồng_thời', ',', 'bệnh_viện', 'tiếp_tục', 'thực_hiện', 'các', 'biện_pháp', 'phòng_chống', 'dịch_bệnh', 'COVID']...
  - Tags: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']...

Thống kê độ dài câu:
  - Min: 6
  - Max: 161
  - Mean: 26.36
  - Median: 24


In [7]:
all_tags = []
for item in train_data:
    all_tags.extend(item['tags'])

unique_tags = sorted(set(all_tags))
print("=" * 60)
print("CÁC NHÃN TRONG DATASET")
print("=" * 60)
print(f"\nTổng số nhãn duy nhất: {len(unique_tags)}")
print(f"\nDanh sách nhãn:")
for tag in unique_tags:
    count = all_tags.count(tag)
    percentage = (count / len(all_tags)) * 100
    print(f"  {tag:25s} : {count:8d} ({percentage:5.2f}%)")

CÁC NHÃN TRONG DATASET

Tổng số nhãn duy nhất: 20

Danh sách nhãn:
  B-AGE                     :      682 ( 0.51%)
  B-DATE                    :     2549 ( 1.92%)
  B-GENDER                  :      542 ( 0.41%)
  B-JOB                     :      205 ( 0.15%)
  B-LOCATION                :     5398 ( 4.07%)
  B-NAME                    :      349 ( 0.26%)
  B-ORGANIZATION            :     1137 ( 0.86%)
  B-PATIENT_ID              :     3240 ( 2.45%)
  B-SYMPTOM_AND_DISEASE     :     1439 ( 1.09%)
  B-TRANSPORTATION          :      226 ( 0.17%)
  I-AGE                     :        2 ( 0.00%)
  I-DATE                    :     2500 ( 1.89%)
  I-JOB                     :       62 ( 0.05%)
  I-LOCATION                :     5242 ( 3.96%)
  I-NAME                    :       13 ( 0.01%)
  I-ORGANIZATION            :     2545 ( 1.92%)
  I-PATIENT_ID              :       11 ( 0.01%)
  I-SYMPTOM_AND_DISEASE     :     1552 ( 1.17%)
  I-TRANSPORTATION          :       67 ( 0.05%)
  O                  

In [8]:
from collections import Counter

# Tách prefix B- và I-
entity_types = {}
for tag in unique_tags:
    if tag == 'O':
        continue
    if tag.startswith('B-'):
        entity_type = tag[2:]
        entity_types[entity_type] = entity_types.get(entity_type, 0) + all_tags.count(tag)

print("=" * 60)
print("CÁC LOẠI ENTITY (chỉ đếm B- tags)")
print("=" * 60)
for entity_type, count in sorted(entity_types.items(), key=lambda x: x[1], reverse=True):
    print(f"  {entity_type:25s} : {count:8d}")

print(f"\nTổng số loại entity: {len(entity_types)}")

CÁC LOẠI ENTITY (chỉ đếm B- tags)
  LOCATION                  :     5398
  PATIENT_ID                :     3240
  DATE                      :     2549
  SYMPTOM_AND_DISEASE       :     1439
  ORGANIZATION              :     1137
  AGE                       :      682
  GENDER                    :      542
  NAME                      :      349
  TRANSPORTATION            :      226
  JOB                       :      205

Tổng số loại entity: 10


In [9]:
print("=" * 60)
print("VÍ DỤ CÁC CÂU CÓ ENTITY")
print("=" * 60)

examples_shown = 0
for item in train_data:
    has_entity = any(tag != 'O' for tag in item['tags'])
    if has_entity and examples_shown < 5:
        print(f"\nCâu {examples_shown + 1}:")
        words = item['words']
        tags = item['tags']
        
        # Highlight entities
        for i, (word, tag) in enumerate(zip(words, tags)):
            if tag != 'O':
                print(f"  [{word}] ({tag})", end=" ")
            else:
                print(word, end=" ")
        print()
        examples_shown += 1

VÍ DỤ CÁC CÂU CÓ ENTITY

Câu 1:
Đồng_thời , bệnh_viện tiếp_tục thực_hiện các biện_pháp phòng_chống dịch_bệnh COVID - 19 theo hướng_dẫn của   [Bộ] (B-ORGANIZATION)   [Y_tế] (I-ORGANIZATION) . 

Câu 2:
" Số bệnh_viện có_thể tiếp_nhận bệnh_nhân bị   [sốt] (B-SYMPTOM_AND_DISEASE)   [cao] (I-SYMPTOM_AND_DISEASE) và   [khó] (B-SYMPTOM_AND_DISEASE)   [thở] (I-SYMPTOM_AND_DISEASE) đang giảm dần " , thông_cáo có đoạn , cảnh_báo những bệnh_nhân này thay vào đó được chuyển tới các phòng_khám khẩn_cấp , khiến những bệnh_nhân mắc bệnh hiểm_nghèo khác không có cơ_hội được điều_trị . 

Câu 3:
Bà này khi trở về quá_cảnh   [Doha] (B-LOCATION) (   [Qatar] (B-LOCATION) ) , đáp xuống   [Tân_Sơn_Nhất] (B-LOCATION) sáng   [2/3] (B-DATE) cùng 75 hành_khách , trong đó có 55 người nước_ngoài . 

Câu 4:
" Bệnh_nhân   [523] (B-PATIENT_ID) " và chồng là " bệnh_nhân   [522] (B-PATIENT_ID) " ,   [67] (B-AGE) tuổi , được   [Bộ] (B-ORGANIZATION)   [Y_tế] (I-ORGANIZATION) ghi_nhận nhiễm nCoV hôm   [31/7] (B-DATE) . 



In [10]:
print("=" * 60)
print("KIỂM TRA CÁC FILE KHÁC")
print("=" * 60)

for filename in ['dev_word.json', 'test_word.json']:
    filepath = f'../dataset/PhoNER/{filename}'
    try:
        data = []
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                data.append(json.loads(line.strip()))
        print(f"\n{filename}:")
        print(f"  - Số câu: {len(data)}")
        
        # Kiểm tra nhãn
        tags_in_file = set()
        for item in data:
            tags_in_file.update(item['tags'])
        print(f"  - Số nhãn duy nhất: {len(tags_in_file)}")
    except FileNotFoundError:
        print(f"\n{filename}: Không tìm thấy file")

KIỂM TRA CÁC FILE KHÁC

dev_word.json:
  - Số câu: 2000
  - Số nhãn duy nhất: 19

test_word.json:
  - Số câu: 3000
  - Số nhãn duy nhất: 20
